# Input

a dataset that includes listings for various areas in Montreal in 2019. It includes detailed information for each listing, such as thumbnails and detailed summarization about each of them.

# Output

predict the listing price based on the listing characteristics, the prices are in categories 0,1,2

# What data mining function is required?

multi-objective (multi-task) multi-modality classification using neural networks

# What could be the challenges?

Developing a successful solution to our problem , complex data,datasets can include complex data elements ,another thing is that we have to make sure that our algorithm must be efficient and scalable to extract information from the big data and we should have enough knowledge and experience in order to use them if we needed to improve our algorithms.

# What is the impact?

Our model predictions is going to implement a listing price based on the listing characteristics which will help people ask for suitable prices on airbnb insteaf of checking prices of similar posts 

# What is an ideal solution?

 An ideal solution in my opinion will be measured in terms of metrics and performances

# What is the experimental protocol used and how was it carried out?

For image data is tackled using convolutional neural network and the text data is processed using embedding and GRU/LSTM layers.

When it comes to preprocessing,  I preprocessed the images by resizing them and converting them to 'LA' , created an array of each images and then returned a new array for 64, 64, 2)  shape, filled with zeroes, loaded images which are in our trainning dataset, identified them using the column 'image' 

for the 'summary' column , I converted some of the non-string cells to string cells        

And I used neural networks in my model

# How did we tune hyper-parameters in the template?

To evaluate hyperparameter combinations, the search function accepts as input training data and a validation split. In random search and Bayesian Optimization, the epochs parameter is used to specify the number of training epochs for each hyperparameter combination.

# What is the search space and what is the criteria to determine good/bad hyper-parameters?

Search space. This component describes the set of possible neural network architectures to consider. These search spaces are designed specific to the application, e.g., a space of convolutional networks for computer vision tasks or a space of recurrent networks for language modeling tasks.

There are basically four methods to determine hyperparameters:

Manual Search: Using knowledge you have about the problem guess parameters and observe the result. 


Grid Search:Determine ranges for the hyperparameters based on your knowledge of the problem. Then choose a few points from those ranges that are usually evenly distributed. Train your network with every possible parameter combination and pick the one that works best. Alternatively, you can refine your search to a more specific domain centred on the best-performing characteristics.


Random Search:To identify ranges for the hyperparameters, you leverage knowledge of the problem in the same way that grid search does. Instead of selecting values from such ranges in a systematic manner, you choose them at random. Use what you learn to reduce your search or repeat this procedure until you find settings that work well. Dr. Bengio suggests this as the baseline approach against which all other methods should be assessed in his paper Random Search for Hyper-Parameter Optimization, and shows that it works better than the others.

Bayesian Optimization:More recent work has focused on improving on these other approaches by deciding how to alter the hyper parameters for the next experiment based on the knowledge acquired from any specific experiment. Adams et alPractical .'s Bayesian Optimization of Machine Learning Algorithms is an example of this type of work.

# Is fully-connected model a good one for sequential data? Why? How about for image data? Is it good? Why?

Deep learning's workhorses, fully connected networks, are employed in thousands of applications. Fully connected networks have the virtue of being "structure agnostic." That is, no specific assumptions about the input are required (for example, that the input consists of images or videos).

Fully Connected Layer is simply, feed forward neural networks.Fully Connected Layers are the network's final layers. The output from the final Pooling or Convolutional Layer, which is flattened and then fed into the fully connected layer, is the input to the fully connected layer. so it's super good for both sequential data & image data!

# What is gradient vanishing and gradient explosion, and how GRU/LSTM tries to mitigate this problem?

n derivatives will be multiplied together in a network of n hidden layers. If the derivatives are significant, the gradient will grow exponentially as we propagate down the model until it explodes, which is referred to as the exploding gradient problem. If the derivatives are tiny, on the other hand, the gradient will fall exponentially as we propagate through the model until it vanishes, which is known as the vanishing gradient problem.

The Sigmoid function saturates at 0 or 1 with a derivative very close to zero for bigger inputs (negative or positive), as shown in the graph above. As a result, when the backpropagation process kicks in, there are almost no gradients to propagate backward in the network, and any residual gradients there are continue to dilute as the programme continues through the top layers. As a result, the bottom layers are left with nothing.

GRU/LSTM:

As a solution to short-term memory, LSTMs and GRUs were developed. They have inbuilt devices known as gates that can control the flow of data.

These gates can figure out which data in a sequence should be kept and which should be discarded. It can then transfer important information down the long chain of sequences to make predictions as a result of this. These two networks are responsible for nearly all state-of-the-art recurrent neural network findings. Voice recognition, speech synthesis, and text production all use LSTMs and GRUs. You can even use them to create video captioning.

# What is multi-objective/multi-task learning? What is multi-modality learning? How do you use them in this assignment?

Multi-task learning: is a machine learning approach in which we try to learn multiple tasks simultaneously, optimizing multiple loss functions at once. Rather than training independent models for each task, we allow a single model to learn to complete all of the tasks at once. In this process, the model uses all of the available data across the different tasks to learn generalized representations of the data that are useful in multiple contexts.

Multi-modality learning: Multimodal learning involves relating information from multiple sources. In our case, we are trying to use the textual data and the image data together to get the required outcome. We make use of the the Multiclass Classification.

We used them in the assignment by predicting both price & type

# What is the difference among xgboost, lightgbm and catboost

Catboost cultivates a well-balanced tree. Leaf-wise (best-first) tree growth is used by LightGBM. It selects to grow the leaf that causes the least amount of loss, allowing the tree to develop in an unbalanced state. When data is small, overfitting might occur because it grows leaf-wise rather than level-wise.

XGBoost was originally produced by University of Washington researchers and is maintained by open-source contributors. Similar to LightGBM, XGBoost uses the gradients of different cuts to select the next cut, but XGBoost also uses the hessian, or second derivative, in its ranking of cuts. Computing this next derivative comes at a slight cost, but it also allows a greater estimation of the cut to use.

# Coding

In [1]:
#first I started importing some necessary libraries
import pandas as pd #I imported pandas cause Using it
#make it way easier when it comes to  processing, analysis and manipulation of data
from tqdm.notebook import tqdm # tqdm derives from the Arabic word taqaddum (تقدّم) which can mean “progress,” Instantly make your loops show a smart progress meter
from PIL import Image # library for image processing.
import os #to be able to use operating system dependant functions
import numpy as np #to deal with arrays
from ast import literal_eval #expands to Abstract Syntax Tree

In [2]:
#reading training dataset
tr=pd.read_csv('train_xy.csv')
tr

,summary,image,type,price
0,"Spacious, sunny and cozy modern apartment in t...",img_train/0.jpg,Apartment,1
1,Located in one of the most vibrant and accessi...,img_train/1.jpg,Apartment,0
2,Logement coquet et douillet à 10 minutes du ce...,img_train/2.jpg,Apartment,1
3,"Beautiful and spacious (1076 sc ft, / 100 mc) ...",img_train/3.jpg,Apartment,1
4,Très grand appartement ''rustique'' et très ag...,img_train/4.jpg,Apartment,0
...,...,...,...,...
7622,"Un grand logement 4 et 1/2, tout inclut, bien ...",img_train/7626.jpg,Apartment,0
7623,Magnificent condo directly on the river. You w...,img_train/7627.jpg,Apartment,2
7624,This apartment is perfect for anyone visiting ...,img_train/7628.jpg,Apartment,1
7625,"It is a cozy ,clean ,and comfortable apartment...",img_train/7629.jpg,Apartment,0


In [3]:
#reading test dataset
ts=pd.read_csv('test_x.csv', index_col = 'id')
ts

,summary,image
id,,
0,Charming warm house is ready to host you here ...,img_test/0.jpg
1,"La chambre est spacieuse et lumineuse, dans un...",img_test/1.jpg
2,Grande chambre confortable située au sous-sol ...,img_test/2.jpg
3,"Près d’un Métro, ligne orange. 10 minutes à pi...",img_test/3.jpg
4,Very bright appartment and very cosy. 2 separa...,img_test/4.jpg
...,...,...
7626,"Large, fully-furnished flat with brick walls a...",img_test/7627.jpg
7627,Logement situé dans le haut d’un duplex. Vivez...,img_test/7628.jpg
7628,"My place is close to parks, . My place is good...",img_test/7629.jpg


In [4]:
tr.shape #displaying shape of training dataset

(7627, 4)

In [5]:
ts.shape #displaying shape of testing dataset

(7360, 2)

In [6]:
tr.describe() #some information about the training dataset such as standard deviation,mean...etc.

,price
count,7627.000000
mean,0.442769
std,0.611946
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,2.000000


In [7]:
ts.describe()#some information about testing dataset

,summary,image
count,7360,7360
unique,6903,7360
top,"Featured in The New York Times, The Wall Stree...",img_test/0.jpg
freq,93,1


In [8]:
# preprocess image data

def load_image(file):
    try:
        # open image file
        image = Image.open(
            file
        # image is converted to 'LA' and is also being resized
        ).convert('LA').resize((64, 64))
        # creating an array of each images
        arr = np.array(image)
    except:
        # returns a new array for given shape below, filled with zeroes
        arr = np.zeros((64, 64, 2))
    return arr


# loading images:
# loads images which are in our trainning dataset, identifies them using the column 'image'.
from tqdm.notebook import tqdm
# Tqdm is a Python library used to display smart progress bars that show the progress of your Python code execution.
x_image = np.array([load_image(i) for i in tqdm(tr.image)])

#  convert some of the non-string cell to string
x_text = tr.summary.astype('str')



  0%|          | 0/7627 [00:00<?, ?it/s]

In [9]:
# labels:
# target column which is price 
y_price = tr.price
#and then encoding our (type) column
y_type = tr.type.astype('category').cat.codes

In [10]:
# calculate the total number of unique labels.
len_price = len(y_price.unique())
len_type = len(y_type.unique())
print('price category number of unique values and unique values are :', len_price, y_price.unique()) #printing for price label
print('type category number of unique values and unique values are :', len_type, y_type.unique())#printing for type label

price category number of unique values and unique values are : 3 [1 0 2]
type category number of unique values and unique values are : 24 [ 1 17 22 10 18 20  5  2  8  4 23 13 15 16 14 11 19  0 21  3  6 12  7  9]


In [11]:
# splitting data into 70% ,30%: (training &validate=ion)
from sklearn.model_selection import train_test_split # function used for data splitting

x_tr_image, x_vl_image, x_tr_text, x_vl_text, y_tr_price, y_vl_price, y_tr_type, y_vl_type = train_test_split(
    x_image, 
    x_text,
    y_price,
    y_type,
    test_size=0.3)


In [12]:
# maximum number of words from the resulting tokenized data which are to be used
# preprocess text data
from tensorflow.keras.preprocessing.text import Tokenizer # to divide strings into lists of substrings
from tensorflow.keras.preprocessing.sequence import pad_sequences #Pad sequences to  same length.
from pprint import pprint #pretty-print Python data structures in a form which can be used as input to the interpreter 
vocab_size = 40000 #assigning vocab size
max_len = 100 # and maximum length


tokenizer = Tokenizer(num_words=vocab_size)
# upadting the internal vocalulary based on the list of text, so it creates the vocabulary
# index based on word frequnecy so every word gets a unique interger value so lower integers 
# mean more frequent word.
tokenizer.fit_on_texts(x_tr_text)


def _preprocess(list_of_text):
    #padding 0 in the beggining of each sequence until they have the same length as
    # the longest sequence. 
    return pad_sequences(
        # texts_to sequence transforms every text in texts to a sequence of integers from dictionary
        tokenizer.texts_to_sequences(list_of_text),
        # takes in the pre-defined input (100) as maximum length of all sequences.
        maxlen=max_len,
        # does padding after each sequence
        padding='post',
    )
    

# padding is done inside: 
x_tr_text_id = _preprocess(x_tr_text)
x_vl_text_id = _preprocess(x_vl_text)

print(x_tr_text_id.shape) #printing for training 
print(x_vl_text_id.shape) #printing for validation

(5338, 100)
(2289, 100)


In [13]:
from tensorflow import keras #Keras is a library that will help us make neural networks.
import tensorflow as tf #open-source library for numerical computation

# defines an input layer ,Input() is used to instantiate a Keras tensor, (instantiate a Keras tensor object) and allows for building a model.
in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 2))


In [14]:
#num_words is  the maximum number of words to keep, based on word frequency.
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(in_text)
#tf.reduce_mean() computes the mean of elements across dimensions of a tensor..
averaged = tf.reduce_mean(embedded, axis=1)

In [15]:
#image part
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D

#Image inputs. Covering at least a Conv2d layer:
# first convolution layer
# the first parameter is the filter indicating the dimensionality of the output space.
# second oe is the kernel_size it tells the height and width of the filter.
cov = Conv2D(32, (16, 16))(in_image)
# Max pooling layer to downsample
pl = MaxPool2D((16, 16))(cov)
# and flattening 
flattened = Flatten()(pl)


# fusion - combining both:
# concatenates tensors along one dimension (axis)
fused = tf.concat([averaged, flattened], axis=-1)
# multi-task learning (each is a multi-class classification)
# one dense layer for each task
# multi-objectives (each is a multi-class classification)
p_price = Dense(len_price, activation='softmax', name='price')(fused)
p_type = Dense(len_type, activation='softmax', name='type')(fused)

# define model input/output using keys.
model = keras.Model(
    inputs={
        'summary': in_text, #the inputs
        'image': in_image
    },
    outputs={ #to be predicted/outputs
        'price': p_price,
        'type': p_type,
    },
)
from tensorflow.keras.optimizers import Adam #as it makes use of a gradient average that decays exponentially over time.

# compile model with optimizer, loss values for each task, loss 
# weights for each task.
model.compile(
    optimizer=Adam(), # Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
    loss={
        'price': 'sparse_categorical_crossentropy',
        'type': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'price': 0.5,
        'type': 0.5,       
    },
    metrics={
        'price': ['SparseCategoricalAccuracy'],
        'type': ['SparseCategoricalAccuracy'],
    },
)


model.summary() #printing the summarzation of our model


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 2)]  0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 49, 49, 32)   16416       ['input_2[0][0]']                
                                                                                                  
 embedding (Embedding)          (None, 100, 100)     4000000     ['input_1[0][0]']                
                                                                                              

In [16]:
history = model.fit( #fiting model
    x={ #inputs
        'summary': x_tr_text_id,
        'image': x_tr_image
    },
    y={ #outputs
        'price': y_tr_price,
        'type': y_tr_type,
    },
    epochs=5, #num of ephocs ( number of passes through the training set) 
    batch_size=16, #numb of samples that will pass to network at a time
    validation_data=(
        {
            'summary': x_vl_text_id,
            'image': x_vl_image
         }, 
        {
            'price': y_vl_price,
            'type': y_vl_type,
        }),
    validation_split=0.3,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_price_loss', patience=5, )
    ],
    verbose=1 #shows progress and a line per each epoch
)

Epoch 1/5
234/234 [==============================] - 102s 406ms/step - loss: 32.3963 - price_loss: 28.0442 - type_loss: 36.7484 - price_sparse_categorical_accuracy: 0.4839 - type_sparse_categorical_accuracy: 0.5699 - val_loss: 13.1139 - val_price_loss: 9.1425 - val_type_loss: 17.0853 - val_price_sparse_categorical_accuracy: 0.6030 - val_type_sparse_categorical_accuracy: 0.7422
Epoch 2/5
234/234 [==============================] - 98s 417ms/step - loss: 10.6004 - price_loss: 8.3771 - type_loss: 12.8238 - price_sparse_categorical_accuracy: 0.5158 - type_sparse_categorical_accuracy: 0.5808 - val_loss: 10.5855 - val_price_loss: 12.2827 - val_type_loss: 8.8883 - val_price_sparse_categorical_accuracy: 0.6136 - val_type_sparse_categorical_accuracy: 0.5481
Epoch 3/5
234/234 [==============================] - 100s 429ms/step - loss: 7.4886 - price_loss: 5.3427 - type_loss: 9.6346 - price_sparse_categorical_accuracy: 0.5161 - type_sparse_categorical_accuracy: 0.5913 - val_loss: 5.0623 - val_price

In [17]:
x_test_summary = _preprocess(ts.summary.astype(str)) 
x_test_image = np.array([load_image(i) for i in tqdm(ts.image)])

  0%|          | 0/7360 [00:00<?, ?it/s]

In [18]:
y_predict = model.predict( #predicting results
    {
        'summary': x_test_summary,
        'image': x_test_image
    }
)

price_predicted = y_predict['price']
print(price_predicted) #predicting price
price_category_predicted = np.argmax(price_predicted, axis=1)
print(price_category_predicted)

pd.DataFrame( #generating submission file
    {'id': ts.index,
     'price': price_category_predicted}).to_csv('sample_submission.csv', index=False)

[[9.9939835e-01 6.0169719e-04 6.7562768e-11]
 [9.9987388e-01 1.2130430e-04 4.7702947e-06]
 [9.9950373e-01 4.9626583e-04 9.8856772e-09]
 ...
 [9.9997127e-01 1.8856001e-05 9.8910423e-06]
 [9.9999487e-01 5.1822353e-06 6.5851101e-13]
 [9.9875045e-01 1.2495047e-03 1.7307313e-09]]
[0 0 0 ... 0 0 0]


# Trial One

Covering a Conv2d layer for Image inputs.

# Expectaion

I expect to get a high accuracy on kaggle after using conv2d layer for image inputs

# Observation

After running my code, my accuracy on kaggle was 0.51793

# Plan

I will use LSTM layer for text inputs and see how my accuracy will get affected

# LSTM

In [20]:
# Long Short Term Memory (LSTM)
# Long Short Term Memory networks – usually just called “LSTMs”
# It can handle not only individual data points like photos, but also speech or video

from tensorflow import keras #Keras is a library that will help us make neural networks.
import tensorflow as tf #open-source library for numerical computation


# defines an input layer ,Input() is used to instantiate a Keras tensor, (instantiate a Keras tensor object) and allows for building a model
in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 2))

# activation determines the output of that node given an input or group of inputs.
# recurrent_activation: Activation function to use for the recurrent step
# dropout: to avoid overfitting in training
# rcurrent_dropout: regularization method for recurrent neural networks.
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(in_text)
lstm = keras.layers.LSTM(124, 
                         activation="tanh",
                         recurrent_activation="sigmoid",
                         return_sequences=True,
                         dropout=0.1,
                         recurrent_dropout=0.1)(embedded)
lstm2 = keras.layers.LSTM(64, 
                         activation="tanh",
                         recurrent_activation="sigmoid",
                         return_sequences=True,
                         dropout=0.1,
                         recurrent_dropout=0.1)(lstm)
lstm3 = keras.layers.LSTM(32, 
                         activation="tanh",
                         recurrent_activation="sigmoid",
                         return_sequences=False,
                         dropout=0.1,
                         recurrent_dropout=0.1)(lstm2)

from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D



# image part
# simple conv2d
# first convolution layer
# the first parameter is the filter indicating the dimensionality of the output space.
# second oe is the kernel_size it tells the height and width of the filter.
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)


# fusion - combining both:
# concatenates tensors along one dimension (axis)
fused = tf.concat([lstm3, flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task
# multi-objectives (each is a multi-class classification)
p_price = Dense(len_price, activation='softmax', name='price')(fused)
p_type = Dense(len_type, activation='softmax', name='type')(fused)

# define model input/output using keys.

model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={ #to be predicted
        'price': p_price,
        'type': p_type,
    },
)

# compile model with optimizer, loss values for each task, loss 
# weights for each task.
from tensorflow.keras.optimizers import Adam #optimizer

model.compile( #compiling the model
    optimizer=Adam(), # Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
    loss={
        'price': 'sparse_categorical_crossentropy',
        'type': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'price': 0.5,
        'type': 0.5,       
    },
    metrics={
        'price': ['SparseCategoricalAccuracy'],
        'type': ['SparseCategoricalAccuracy'],
    },
)

model.summary() #printing summarization

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 100, 100)     4000000     ['input_5[0][0]']                
                                                                                                  
 input_6 (InputLayer)           [(None, 64, 64, 2)]  0           []                               
                                                                                                  
 lstm_3 (LSTM)                  (None, 100, 124)     111600      ['embedding_2[0][0]']            
                                                                                            

In [21]:
history = model.fit( #fiting model
    x={ #inputs
        'summary': x_tr_text_id,
        'image': x_tr_image
    },
    y={ #outputs
        'price': y_tr_price,
        'type': y_tr_type,
    },
    epochs=5, #num of ephocs ( number of passes through the training set) 
    batch_size=16, #numb of samples that will pass to network at a time
    validation_data=(
        {
            'summary': x_vl_text_id,
            'image': x_vl_image
         }, 
        {
            'price': y_vl_price,
            'type': y_vl_type,
        }),
    validation_split=0.3,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_price_loss', patience=5, )
    ],
    verbose=1 #shows progress and a line per each epoch
)

Epoch 1/5
234/234 [==============================] - 330s 1s/step - loss: 35.1297 - price_loss: 34.6147 - type_loss: 35.6446 - price_sparse_categorical_accuracy: 0.4920 - type_sparse_categorical_accuracy: 0.5720 - val_loss: 21.4304 - val_price_loss: 28.5235 - val_type_loss: 14.3373 - val_price_sparse_categorical_accuracy: 0.3221 - val_type_sparse_categorical_accuracy: 0.6692
Epoch 2/5
234/234 [==============================] - 292s 1s/step - loss: 12.7582 - price_loss: 10.6210 - type_loss: 14.8953 - price_sparse_categorical_accuracy: 0.5142 - type_sparse_categorical_accuracy: 0.5779 - val_loss: 17.5407 - val_price_loss: 22.4734 - val_type_loss: 12.6080 - val_price_sparse_categorical_accuracy: 0.1155 - val_type_sparse_categorical_accuracy: 0.7628
Epoch 3/5
234/234 [==============================] - 298s 1s/step - loss: 7.8175 - price_loss: 5.9612 - type_loss: 9.6738 - price_sparse_categorical_accuracy: 0.5423 - type_sparse_categorical_accuracy: 0.5851 - val_loss: 9.4962 - val_price_loss

In [22]:
x_test_summary = _preprocess(ts.summary.astype(str)) 
x_test_image = np.array([load_image(i) for i in tqdm(ts.image)])

  0%|          | 0/7360 [00:00<?, ?it/s]

In [23]:
y_predict = model.predict( #predicting results
    {
        'summary': x_test_summary,
        'image': x_test_image
    }
)

price_predicted = y_predict['price']
print(price_predicted) #predicting price
price_category_predicted = np.argmax(price_predicted, axis=1)
print(price_category_predicted)

pd.DataFrame( #generating submission file
    {'id': ts.index,
     'price': price_category_predicted}).to_csv('sample_submission.csv', index=False)

[[9.99991536e-01 8.18381613e-06 2.66567611e-07]
 [9.99988914e-01 1.09100247e-05 1.53425063e-07]
 [8.97141039e-01 6.78245574e-02 3.50343995e-02]
 ...
 [9.93793249e-01 1.24735625e-05 6.19432237e-03]
 [6.35580858e-03 1.38107187e-03 9.92263079e-01]
 [9.98858094e-01 1.11520337e-03 2.67184005e-05]]
[0 0 0 ... 0 2 0]


# Trial two

using LSTM layer for text inputs

# Expectaion

I expect to get a high accuracy on kaggle after using LSTM layer for text inputs

# Observation

After running my code, my accuracy on kaggle dropped to 0.43369 :(

# Plan

I will use a BiDirectional layer for text inputs

#  BiDirectional layer

In [24]:
# image part
# simple conv2d
# first convolution layer
# the first parameter is the filter indicating the dimensionality of the output space.
# second oe is the kernel_size it tells the height and width of the filter.
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)


# fusion - combining both:
# concatenates tensors along one dimension (axis)
# Bidirectional RNN
# Bidirectional recurrent neural networks(RNN) are really just putting two independent RNNs together.
# This structure allows the networks to have both backward and forward
# information about the sequence at every time step.

from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.optimizers import Adam

in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 2))

# Bidirectional Layer:
# Bidirectional LSTMs are an extension of traditional LSTMs that can improve model
# performance on sequence classification problems.
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(in_text)
bidirectional = keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True))(embedded)
bidirectional1 = keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True))(bidirectional)
bidirectional2 = keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=False))(bidirectional1)

# image part
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)

# fusion:
fused = tf.concat([bidirectional2, flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task
# multi-objectives (each is a multi-class classification)
p_price = Dense(len_price, activation='softmax', name='price')(fused)
p_type = Dense(len_type, activation='softmax', name='type')(fused)

# define model input/output using keys.
import tensorflow as tf
from tensorflow import keras

model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={ #to be predicted
        'price': p_price,
        'type': p_type,
    },
)

# compile model with optimizer, loss values for each task, loss 
# weights for each task.
from tensorflow.keras.optimizers import Adam #optimizer

model.compile(
    optimizer=Adam(), # Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
    loss={
        'price': 'sparse_categorical_crossentropy',
        'type': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'price': 0.5,
        'type': 0.5,       
    },
    metrics={
        'price': ['SparseCategoricalAccuracy'],
        'type': ['SparseCategoricalAccuracy'],
    },
)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 100, 100)     4000000     ['input_7[0][0]']                
                                                                                                  
 input_8 (InputLayer)           [(None, 64, 64, 2)]  0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 100, 200)     160800      ['embedding_3[0][0]']            
                                                                                            

In [25]:
history = model.fit( #fiting model
    x={ #inputs
        'summary': x_tr_text_id,
        'image': x_tr_image
    },
    y={ #outputs
        'price': y_tr_price,
        'type': y_tr_type,
    },
    epochs=5, #num of ephocs ( number of passes through the training set) 
    batch_size=16, #numb of samples that will pass to network at a time
    validation_data=(
        {
            'summary': x_vl_text_id,
            'image': x_vl_image
         }, 
        {
            'price': y_vl_price,
            'type': y_vl_type,
        }),
    validation_split=0.3,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_price_loss', patience=5, )
    ],
    verbose=1 #shows progress and a line per each epoch
)

Epoch 1/5
234/234 [==============================] - 347s 1s/step - loss: 29.2694 - price_loss: 24.9294 - type_loss: 33.6094 - price_sparse_categorical_accuracy: 0.5059 - type_sparse_categorical_accuracy: 0.5715 - val_loss: 11.9814 - val_price_loss: 7.2505 - val_type_loss: 16.7123 - val_price_sparse_categorical_accuracy: 0.5362 - val_type_sparse_categorical_accuracy: 0.1654
Epoch 2/5
234/234 [==============================] - 329s 1s/step - loss: 8.1705 - price_loss: 6.3059 - type_loss: 10.0352 - price_sparse_categorical_accuracy: 0.5278 - type_sparse_categorical_accuracy: 0.5779 - val_loss: 6.2277 - val_price_loss: 4.4000 - val_type_loss: 8.0554 - val_price_sparse_categorical_accuracy: 0.5968 - val_type_sparse_categorical_accuracy: 0.6848
Epoch 3/5
234/234 [==============================] - 333s 1s/step - loss: 10.2234 - price_loss: 6.7054 - type_loss: 13.7413 - price_sparse_categorical_accuracy: 0.5476 - type_sparse_categorical_accuracy: 0.5840 - val_loss: 7.3341 - val_price_loss: 5.

In [26]:
x_test_summary = _preprocess(ts.summary.astype(str)) 
x_test_image = np.array([load_image(i) for i in tqdm(ts.image)])

  0%|          | 0/7360 [00:00<?, ?it/s]

In [27]:
y_predict = model.predict( #predicting results
    {
        'summary': x_test_summary,
        'image': x_test_image
    }
)

price_predicted = y_predict['price']
print(price_predicted) #predicting price
price_category_predicted = np.argmax(price_predicted, axis=1)
print(price_category_predicted)

pd.DataFrame( #generating submission file
    {'id': ts.index,
     'price': price_category_predicted}).to_csv('sample_submission.csv', index=False)

[[1.0000000e+00 5.2564827e-11 1.4762740e-10]
 [9.9996305e-01 3.6965408e-05 4.5654102e-11]
 [1.0000000e+00 4.4944084e-08 9.0852923e-16]
 ...
 [1.2692772e-02 9.8730719e-01 7.8246767e-12]
 [4.0130215e-03 9.9598700e-01 5.8214953e-17]
 [9.9810398e-01 1.8960247e-03 1.7611444e-11]]
[0 0 0 ... 1 1 0]


# Trial three

using BiDirectional layer for text inputs

# Expectaion

I expect to get a high accuracy on kaggle after using BiDirectional layer for text inputs

# observation 

After running my code, my accuracy on kaggle increased to 0.48423

# Plan

I will use BiDirectional layer for text inputs and Dropout layer for image inputs


# BiDirectional layer for text inputs and Dropout layer for image inputs

In [28]:
# fusion - combining both:
# concatenates tensors along one dimension (axis)
# Bidirectional RNN
# Bidirectional recurrent neural networks(RNN) are really just putting two independent RNNs together.
# This structure allows the networks to have both backward and forward
# information about the sequence at every time step.

from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.optimizers import Adam

in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 2))

In [29]:
# Bidirectional Layer:
# Bidirectional LSTMs are an extension of traditional LSTMs that can improve model
# performance on sequence classification problems.
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(in_text)
bidirectional = keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True))(embedded)
bidirectional1 = keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True))(bidirectional)
bidirectional2 = keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=False))(bidirectional1)

# image part
drop_1 = tf.keras.layers.Dropout(.1) #dropout layer specification

cov1 = Conv2D(45, (2, 2))(in_image) #2D Convolutional layer with 32 filters and filter size of 16x16 Output:(None, 63, 63, 45)
outputs = drop_1(cov1, training=True) #(None, 63, 63, 45) 
p1 = MaxPool2D((3, 3))(outputs) #(None, 21, 21, 45)
#cov2 = Conv2D(16, (2, 2))(p1)
#p2 = MaxPool2D((2, 2))(cov2)        
flattened = Flatten()(p1)            #flattening layer Output Shape (None, 19845) 
fc2 = tf.keras.layers.Dense(200, activation='relu')(flattened)

# fusion:
fused = tf.concat([bidirectional2, flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task
# multi-objectives (each is a multi-class classification)
p_price = Dense(len_price, activation='softmax', name='price')(fused)
p_type = Dense(len_type, activation='softmax', name='type')(fused)

# define model input/output using keys.
import tensorflow as tf
from tensorflow import keras

model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={ #to be predicted
        'price': p_price,
        'type': p_type,
    },
)

# compile model with optimizer, loss values for each task, loss 
# weights for each task.
from tensorflow.keras.optimizers import Adam #optimizer

model.compile(
    optimizer=Adam(), # Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
    loss={
        'price': 'sparse_categorical_crossentropy',
        'type': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'price': 0.5,
        'type': 0.5,       
    },
    metrics={
        'price': ['SparseCategoricalAccuracy'],
        'type': ['SparseCategoricalAccuracy'],
    },
)

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 64, 64, 2)]  0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 100, 100)     4000000     ['input_9[0][0]']                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 63, 63, 45)   405         ['input_10[0][0]']               
                                                                                            

In [30]:
history = model.fit( #fiting model
    x={ #inputs
        'summary': x_tr_text_id,
        'image': x_tr_image
    },
    y={ #outputs
        'price': y_tr_price,
        'type': y_tr_type,
    },
    epochs=5, #num of ephocs ( number of passes through the training set) 
    batch_size=16, #numb of samples that will pass to network at a time
    validation_data=(
        {
            'summary': x_vl_text_id,
            'image': x_vl_image
         }, 
        {
            'price': y_vl_price,
            'type': y_vl_type,
        }),
    validation_split=0.3,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_price_loss', patience=5, )
    ],
    verbose=1 #shows progress and a line per each epoch
)

Epoch 1/5
234/234 [==============================] - 376s 1s/step - loss: 57.1222 - price_loss: 40.6415 - type_loss: 73.6030 - price_sparse_categorical_accuracy: 0.5037 - type_sparse_categorical_accuracy: 0.5691 - val_loss: 13.5224 - val_price_loss: 11.7410 - val_type_loss: 15.3038 - val_price_sparse_categorical_accuracy: 0.3964 - val_type_sparse_categorical_accuracy: 0.5044
Epoch 2/5
234/234 [==============================] - 295s 1s/step - loss: 8.7142 - price_loss: 7.4767 - type_loss: 9.9517 - price_sparse_categorical_accuracy: 0.5399 - type_sparse_categorical_accuracy: 0.6081 - val_loss: 9.7338 - val_price_loss: 7.4987 - val_type_loss: 11.9689 - val_price_sparse_categorical_accuracy: 0.4875 - val_type_sparse_categorical_accuracy: 0.6042
Epoch 3/5
234/234 [==============================] - 296s 1s/step - loss: 5.1792 - price_loss: 4.9085 - type_loss: 5.4500 - price_sparse_categorical_accuracy: 0.5806 - type_sparse_categorical_accuracy: 0.6555 - val_loss: 7.9155 - val_price_loss: 6.5

In [31]:
x_test_summary = _preprocess(ts.summary.astype(str)) 
x_test_image = np.array([load_image(i) for i in tqdm(ts.image)])

  0%|          | 0/7360 [00:00<?, ?it/s]

In [32]:
y_predict = model.predict( #predicting results
    {
        'summary': x_test_summary,
        'image': x_test_image
    }
)

price_predicted = y_predict['price']
print(price_predicted) #predicting price
price_category_predicted = np.argmax(price_predicted, axis=1)
print(price_category_predicted)

pd.DataFrame( #generating submission file
    {'id': ts.index,
     'price': price_category_predicted}).to_csv('sample_submission.csv', index=False)

[[1.5635708e-01 8.4364289e-01 7.8341578e-15]
 [4.7073895e-01 5.2926099e-01 1.2623823e-11]
 [9.9974149e-01 2.5852400e-04 1.8917454e-11]
 ...
 [4.3199950e-09 1.0000000e+00 6.7965083e-14]
 [9.9998224e-01 1.7753038e-05 2.6690757e-16]
 [3.1571828e-03 9.9684227e-01 6.5280989e-07]]
[1 1 0 ... 1 0 1]


# Trial four

I tried using BiDirectional layer for text inputs and Dropout layer for image inputs

# Expectation

I expected to get a high kaggle accuracy after using BiDirectional layer for text inputs and Dropout layer for image inputs

# observation 

My kaggle accuracy increased to 0.55543

# Plan

I will use LSTM layer for text inputs & dropout layer for image inputs

# LSTM layer for text inputs & dropout layer for image inputs

In [33]:
# Long Short Term Memory (LSTM)
# Long Short Term Memory networks – usually just called “LSTMs”
# It can handle not only individual data points like photos, but also speech or video

from tensorflow import keras #Keras is a library that will help us make neural networks.
import tensorflow as tf #open-source library for numerical computation


# defines an input layer ,Input() is used to instantiate a Keras tensor, (instantiate a Keras tensor object) and allows for building a model
in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 2))

# activation determines the output of that node given an input or group of inputs.
# recurrent_activation: Activation function to use for the recurrent step
# dropout: to avoid overfitting in training
# rcurrent_dropout: regularization method for recurrent neural networks.
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(in_text)
lstm = keras.layers.LSTM(124, 
                         activation="tanh",
                         recurrent_activation="sigmoid",
                         return_sequences=True,
                         dropout=0.1,
                         recurrent_dropout=0.1)(embedded)
lstm2 = keras.layers.LSTM(64, 
                         activation="tanh",
                         recurrent_activation="sigmoid",
                         return_sequences=True,
                         dropout=0.1,
                         recurrent_dropout=0.1)(lstm)
lstm3 = keras.layers.LSTM(32, 
                         activation="tanh",
                         recurrent_activation="sigmoid",
                         return_sequences=False,
                         dropout=0.1,
                         recurrent_dropout=0.1)(lstm2)

from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D



# image part
drop_1 = tf.keras.layers.Dropout(.1) #dropout layer specification

cov1 = Conv2D(45, (2, 2))(in_image) #2D Convolutional layer with 32 filters and filter size of 16x16 Output:(None, 63, 63, 45)
outputs = drop_1(cov1, training=True) #(None, 63, 63, 45) 
p1 = MaxPool2D((3, 3))(outputs) #(None, 21, 21, 45)
#cov2 = Conv2D(16, (2, 2))(p1)
#p2 = MaxPool2D((2, 2))(cov2)        
flattened = Flatten()(p1)            #flattening layer Output Shape (None, 19845) 
fc2 = tf.keras.layers.Dense(200, activation='relu')(flattened)



# fusion - combining both:
# concatenates tensors along one dimension (axis)
fused = tf.concat([lstm3, flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task
# multi-objectives (each is a multi-class classification)
p_price = Dense(len_price, activation='softmax', name='price')(fused)
p_type = Dense(len_type, activation='softmax', name='type')(fused)

# define model input/output using keys.

model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={ #to be predicted
        'price': p_price,
        'type': p_type,
    },
)

# compile model with optimizer, loss values for each task, loss 
# weights for each task.
from tensorflow.keras.optimizers import Adam #optimizer

model.compile(
    optimizer=Adam(), # Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
    loss={
        'price': 'sparse_categorical_crossentropy',
        'type': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'price': 0.5,
        'type': 0.5,       
    },
    metrics={
        'price': ['SparseCategoricalAccuracy'],
        'type': ['SparseCategoricalAccuracy'],
    },
)

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 64, 64, 2)]  0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 100, 100)     4000000     ['input_11[0][0]']               
                                                                                                  
 conv2d_6 (Conv2D)              (None, 63, 63, 45)   405         ['input_12[0][0]']               
                                                                                            

In [34]:
history = model.fit( #fiting model
    x={ #inputs
        'summary': x_tr_text_id,
        'image': x_tr_image
    },
    y={ #outputs
        'price': y_tr_price,
        'type': y_tr_type,
    },
    epochs=5, #num of ephocs ( number of passes through the training set) 
    batch_size=16, #numb of samples that will pass to network at a time
    validation_data=(
        {
            'summary': x_vl_text_id,
            'image': x_vl_image
         }, 
        {
            'price': y_vl_price,
            'type': y_vl_type,
        }),
    validation_split=0.3,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_price_loss', patience=5, )
    ],
    verbose=1 #shows progress and a line per each epoch
)

Epoch 1/5
234/234 [==============================] - 439s 1s/step - loss: 45.6848 - price_loss: 36.4765 - type_loss: 54.8931 - price_sparse_categorical_accuracy: 0.5040 - type_sparse_categorical_accuracy: 0.5728 - val_loss: 11.9748 - val_price_loss: 10.6857 - val_type_loss: 13.2640 - val_price_sparse_categorical_accuracy: 0.5031 - val_type_sparse_categorical_accuracy: 0.4969
Epoch 2/5
234/234 [==============================] - 286s 1s/step - loss: 8.4095 - price_loss: 7.0200 - type_loss: 9.7990 - price_sparse_categorical_accuracy: 0.5522 - type_sparse_categorical_accuracy: 0.6004 - val_loss: 8.4847 - val_price_loss: 6.7977 - val_type_loss: 10.1717 - val_price_sparse_categorical_accuracy: 0.4419 - val_type_sparse_categorical_accuracy: 0.4732
Epoch 3/5
234/234 [==============================] - 289s 1s/step - loss: 5.2012 - price_loss: 4.9381 - type_loss: 5.4643 - price_sparse_categorical_accuracy: 0.5720 - type_sparse_categorical_accuracy: 0.6448 - val_loss: 7.1095 - val_price_loss: 6.0

In [35]:
x_test_summary = _preprocess(ts.summary.astype(str)) 
x_test_image = np.array([load_image(i) for i in tqdm(ts.image)])

  0%|          | 0/7360 [00:00<?, ?it/s]

In [36]:
y_predict = model.predict( #predicting results
    {
        'summary': x_test_summary,
        'image': x_test_image
    }
)

price_predicted = y_predict['price']
print(price_predicted) #predicting price
price_category_predicted = np.argmax(price_predicted, axis=1)
print(price_category_predicted)

pd.DataFrame( #generating submission file
    {'id': ts.index,
     'price': price_category_predicted}).to_csv('sample_submission.csv', index=False)

[[1.00000000e+00 1.52129420e-11 3.23245255e-14]
 [1.00000000e+00 3.67266786e-08 8.80627544e-15]
 [9.99992251e-01 5.62850211e-09 7.72380736e-06]
 ...
 [1.00000000e+00 3.46860562e-09 1.79525549e-16]
 [9.99987245e-01 1.27010735e-05 9.14231624e-12]
 [9.99929428e-01 6.02013170e-05 1.04160254e-05]]
[0 0 0 ... 0 0 0]


# Trial five 

I tried using # LSTM layer for text inputs & dropout layer for image inputs

# Expectation

I expected getting a high kaggle accuracy after using LSTM layer for text inputs & dropout layer for image inputs

# observation

my accuracy dropped to 0.50489

# Plan

I will use GRU layer for text input

In [37]:
# Gated Recurrent Unit (GRU)
# The structure of the GRU allows it to adaptively capture dependencies
# from large sequences of data without discarding information from earlier
# parts of the sequence. This is achieved through its gating units.

from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.optimizers import Adam

in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 2))

# GRU paramerters:
# units: Positive integer, dimensionality of the output space.
# activation: Activation function to use. Default: hyperbolic tangent (tanh). If you pass None, no activation is applied (ie. "linear" activation: a(x) = x).
# recurrent_activation: Activation function to use for the recurrent step. Default: sigmoid (sigmoid). If you pass None, no activation is applied (ie. "linear" activation: a(x) = x).
# dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the inputs. Default: 0.
# recurrent_dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the recurrent state. Default: 0.
# return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence. Default: False.
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(in_text)
gru = keras.layers.GRU(124, 
                       activation="tanh",
                       recurrent_activation="sigmoid",
                       return_sequences=True,
                       dropout=0.1,
                       recurrent_dropout=0.1)(embedded)
gru2 = keras.layers.GRU(64, 
                        activation="tanh",
                        recurrent_activation="sigmoid",
                        return_sequences=True,
                        dropout=0.1,
                        recurrent_dropout=0.1)(gru)
gru3 = keras.layers.GRU(32, 
                        activation="tanh",
                        recurrent_activation="sigmoid",
                        return_sequences=False,
                        dropout=0.1,
                        recurrent_dropout=0.1)(gru2)

# image part
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)

# fusion:
fused = tf.concat([gru3, flattened], axis=-1)

# multi-objectives (each is a multi-class classification)
p_price = Dense(len_price, activation='softmax', name='price')(fused)
p_type = Dense(len_type, activation='softmax', name='type')(fused)


model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={
        'price': p_price,
        'type': p_type,
    },
)


model.compile(
    optimizer=Adam(),
    loss={
        'price': 'sparse_categorical_crossentropy',
        'type': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'price': 0.5,
        'type': 0.5,       
    },
    metrics={
        'price': ['SparseCategoricalAccuracy'],
        'type': ['SparseCategoricalAccuracy'],
    },
)


model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 100, 100)     4000000     ['input_13[0][0]']               
                                                                                                  
 input_14 (InputLayer)          [(None, 64, 64, 2)]  0           []                               
                                                                                                  
 gru (GRU)                      (None, 100, 124)     84072       ['embedding_6[0][0]']            
                                                                                            

In [38]:
history = model.fit( #fiting model
    x={ #inputs
        'summary': x_tr_text_id,
        'image': x_tr_image
    },
    y={ #outputs
        'price': y_tr_price,
        'type': y_tr_type,
    },
    epochs=5, #num of ephocs ( number of passes through the training set) 
    batch_size=16, #numb of samples that will pass to network at a time
    validation_data=(
        {
            'summary': x_vl_text_id,
            'image': x_vl_image
         }, 
        {
            'price': y_vl_price,
            'type': y_vl_type,
        }),
    validation_split=0.3,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_price_loss', patience=5, )
    ],
    verbose=1 #shows progress and a line per each epoch
)

Epoch 1/5
234/234 [==============================] - 346s 1s/step - loss: 27.7542 - price_loss: 23.7706 - type_loss: 31.7379 - price_sparse_categorical_accuracy: 0.4957 - type_sparse_categorical_accuracy: 0.5650 - val_loss: 10.0212 - val_price_loss: 6.4730 - val_type_loss: 13.5695 - val_price_sparse_categorical_accuracy: 0.5593 - val_type_sparse_categorical_accuracy: 0.6242
Epoch 2/5
234/234 [==============================] - 275s 1s/step - loss: 10.9991 - price_loss: 8.0523 - type_loss: 13.9459 - price_sparse_categorical_accuracy: 0.5150 - type_sparse_categorical_accuracy: 0.5715 - val_loss: 7.3369 - val_price_loss: 4.9883 - val_type_loss: 9.6855 - val_price_sparse_categorical_accuracy: 0.5337 - val_type_sparse_categorical_accuracy: 0.6542
Epoch 3/5
234/234 [==============================] - 275s 1s/step - loss: 9.5719 - price_loss: 7.1899 - type_loss: 11.9539 - price_sparse_categorical_accuracy: 0.5134 - type_sparse_categorical_accuracy: 0.5819 - val_loss: 8.1958 - val_price_loss: 6.

In [39]:
x_test_summary = _preprocess(ts.summary.astype(str)) 
x_test_image = np.array([load_image(i) for i in tqdm(ts.image)])

  0%|          | 0/7360 [00:00<?, ?it/s]

In [40]:
y_predict = model.predict( #predicting results
    {
        'summary': x_test_summary,
        'image': x_test_image
    }
)

price_predicted = y_predict['price']
print(price_predicted) #predicting price
price_category_predicted = np.argmax(price_predicted, axis=1)
print(price_category_predicted)

pd.DataFrame( #generating submission file
    {'id': ts.index,
     'price': price_category_predicted}).to_csv('sample_submission.csv', index=False)

[[1.0000000e+00 4.4918269e-09 5.1939990e-13]
 [3.6302037e-04 9.9963701e-01 2.3723864e-08]
 [1.0000000e+00 6.0809198e-09 7.1697899e-13]
 ...
 [1.0000000e+00 4.5051149e-10 6.6938327e-10]
 [9.9561018e-01 4.3898080e-03 5.1747978e-10]
 [9.9998283e-01 1.3732498e-05 3.4867060e-06]]
[0 1 0 ... 0 0 0]


# Trial six

I tried using GRU Layer for text inputs

# Expectation

I expected getting a higher accuracy on kaggle

# Observation

My kaggle accuracy dropped again to 0.38369

# Resouces

https://pypi.org/project/tqdm/

https://www.aipython.in/python-literal_eval/

https://keras.io/api/optimizers/adam/

https://www.bing.com/search?q=What+is+the+search+space+in+deep+learning&go=Search&qs=ds&form=QBRE

https://stats.stackexchange.com/questions/95495/guideline-to-select-the-hyperparameters-in-deep-learning

https://www.oreilly.com/library/view/tensorflow-for-deep/9781491980446/ch04.html

https://www.analyticsvidhya.com/blog/2021/06/the-challenge-of-vanishing-exploding-gradients-in-deep-neural-networks/

https://towardsdatascience.com/the-vanishing-exploding-gradient-problem-in-deep-neural-networks-191358470c11

https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

https://www.springboard.com/blog/data-science/xgboost-random-forest-catboost-lightgbm/